In [1]:
import numpy as np 
import pandas as pd

df = pd.read_csv('E:\\SkillFactory\\datatone\\SolarPrediction.csv')

df['month'] = pd.to_datetime(df['Data']).dt.month
df['day'] = pd.to_datetime(df['Data']).dt.day

df['hour'] = pd.to_datetime(df['Time']).dt.hour
df['minute'] = pd.to_datetime(df['Time']).dt.minute
df['second'] = pd.to_datetime(df['Time']).dt.second

df['rise_hour'] = pd.to_datetime(df['TimeSunRise']).dt.hour
df['rise_minute'] = pd.to_datetime(df['TimeSunRise']).dt.minute

df['set_hour'] = pd.to_datetime(df['TimeSunSet']).dt.hour
df['set_minute'] = pd.to_datetime(df['TimeSunSet']).dt.minute

df.drop(['UNIXTime', 'Data', 'Time', 'TimeSunRise', 'TimeSunSet'], axis = 1, inplace = True)

def get_day(row):
    if row['rise_hour'] < row['hour'] < row['set_hour']:
        return 1
    else:
        return 0
df['day'] = df.apply(get_day, axis=1) # создание нового признака, когда время наблюдения в период с восхода до захода солнца
df_day = df[df['day']==1]

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle

X = df_day[['Temperature', 'Pressure','Humidity', 'WindDirection(Degrees)', 'Speed', 'hour']]
y = np.log(df_day['Radiation'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cbr_scaler = StandardScaler()
X_train = cbr_scaler.fit_transform(X_train)

with open('cbr_scaler.pkl', 'wb') as scaler_file:
    pickle.dump(cbr_scaler, scaler_file)

C:\Users\User\AppData\Local\Temp\ipykernel_7552\4065295809.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month'] = pd.to_datetime(df['Data']).dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_7552\4065295809.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['day'] = pd.to_datetime(df['Data']).dt.day
C:\Users\User\AppData\Local\Temp\ipykernel_7552\4065295809.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['Time']).dt.hour
C:\Users\User\AppData\Local\Temp\ipykernel_7552\4065295809.py:10: UserWarning: Could not infer format, 

In [ ]:
X_test = cbr_scaler.transform(X_test)
from catboost import CatBoostRegressor
cbr_model = CatBoostRegressor(colsample_bylevel=1.0, learning_rate=0.1, max_depth=10, n_estimators=200, reg_lambda=3.0, subsample=1.0)
cbr_model.fit(X_train, y_train)

with open('cbr_model.pkl', 'wb') as model_file:
    pickle.dump(cbr_model, model_file)